In [28]:
## Loading relevant packages
library(slackr)
library(AirSensor)
library(MazamaSpatialUtils)
library(lubridate)
library(dplyr)

## Loading Spatial data 
## More details on Air Sensor Git hub page if it's not setup already
initializeMazamaSpatialUtils("/home/ubuntu/Git/Sensor_Alert/Spatial_data")

## Settng the Country Code API link and look back days 
pas <- pas_createNew(countryCodes = "IN",
                     baseUrl = 'https://www.purpleair.com/json?all=true',lookbackDays = 30,
                     includePWFSL = F)
## Changing the time zone to Asia/Kolkata
pas$lastSeenDate <- with_tz(pas$lastSeenDate,tz="Asia/Kolkata")



In [29]:
SAMOSA_pas = pas%>% pas_filter(!grepl("B",label),grepl("SAMOSA",label))
Wifi = read.csv("/home/ubuntu/Git/Sensor_Alert/Data/WIFI_SAMOSA.csv",col.names=c("label","Dongle"))
tail(SAMOSA_pas)


ID,label,DEVICE_LOCATIONTYPE,THINGSPEAK_PRIMARY_ID,THINGSPEAK_PRIMARY_ID_READ_KEY,THINGSPEAK_SECONDARY_ID,THINGSPEAK_SECONDARY_ID_READ_KEY,latitude,longitude,pm25,⋯,deviceID,locationID,deviceDeploymentID,airDistrict,pwfsl_closestDistance,pwfsl_closestMonitorID,sensorManufacturer,targetPollutant,technologyType,communityRegion
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
48935,SAMOSA_0167,outside,1000402,G2W91CM7BM21Y8OF,1000403,TQ1CKAMEHMT9JG0Z,28.54651,77.19185,46.66,⋯,48935,3b7623508e96caee,3b7623508e96caee_48935,NA,NA,NA,Purple Air,PM,consumer-grade,NA
48871,SAMOSA_0168,outside,1000201,KDEF9H2K7QB7XV1C,1000202,VS3SACHOGSHHJ9J8,28.54583,77.19203,47.40,⋯,48871,5965d2a6f1f21dbd,5965d2a6f1f21dbd_48871,NA,NA,NA,Purple Air,PM,consumer-grade,NA
48981,SAMOSA_0171,outside,1000563,VPT1OVL1038Q9NMH,1000564,QFMIW0JYEDSFIEFT,28.54673,77.19125,20.75,⋯,48981,1a2339b1e6b90275,1a2339b1e6b90275_48981,NA,NA,NA,Purple Air,PM,consumer-grade,NA
48853,SAMOSA_0172,outside,1000165,9FG8QY9A48AXDPBH,1000166,PSK6LR1RNPBPEAUQ,28.54763,77.19039,22.93,⋯,48853,c13fbe948cb73979,c13fbe948cb73979_48853,NA,NA,NA,Purple Air,PM,consumer-grade,NA
48869,SAMOSA_0173,outside,1000197,4O2OC2I7PQ4CZESL,1000198,54J55ZRBZ99NT9NL,28.54684,77.19091,47.60,⋯,48869,e7bc2f3035de3615,e7bc2f3035de3615_48869,NA,NA,NA,Purple Air,PM,consumer-grade,NA
49015,SAMOSA_0174,outside,1001471,ALUYKXPSPGAV0UQV,1001472,1964VNYXPQ1OK05V,28.54559,77.19117,21.80,⋯,49015,8ffe0555392f518b,8ffe0555392f518b_49015,NA,NA,NA,Purple Air,PM,consumer-grade,NA


In [30]:
#Send notification to slack channel
time <- Sys.time()
## Changing the time zone to UTC
time <- with_tz(time,tz="Asia/Kolkata")
## Creating one hour lag for the test (If any sensor is inactive for more than an hour we send an alert)
time_check = time-minutes(5)
## Finding any inactive sensors
Inactive_sensors = SAMOSA_pas %>% filter(time_check>lastSeenDate) %>% select(label,lastSeenDate)%>%
left_join(Wifi,by="label")
dim(Inactive_sensors)[1]

[1] 83

In [31]:
## Adding the R environment file. This file contains the API used for analysis
readRenviron("/home/ubuntu/Git/API.Renviron")

slackr_setup(channel = "pa_sensor_alert",
             bot_user_oauth_token =  Sys.getenv('bot_user_oauth_token'),
             incoming_webhook_url = Sys.getenv('incoming_webhook_url'))

[1] "Successfully connected to Slack"

In [38]:
if (dim(Inactive_sensors)[1] > 0){
    
    slackr("Time of Query",time)
    slackr("Total Inactive sensors", dim(Inactive_sensors)[1])
    ## Sending the Slack alert if the sensors is inactive at any moment
    slackr("Name of Inactive sensors at the moment", pander::pandoc.table(nrow(Inactive_sensors)))
    slackr("Name of Inactive sensors at the moment", pander::pandoc.table(Inactive_sensors))
}